In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install levenshtein

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#importing the dataset
import pandas as pd
rasp_data = pd.read_csv("/content/drive/Shareddrives/2:1 Caitlin & Kimai/Data/Enzyme Stability Prediction/cavity_pred_CUSTOM_A.csv")
rasp_data.head()

,pdbid,chainid,variant,wt_idx,mt_idx,wt_nlf,mt_nlf,score_ml_fermi,score_ml,pos,wt_AA,mt_AA,wt
0,CUSTOM,A,V1A,17,0,2.623741,2.483982,0.240600,0.126521,1,V,A,V
1,CUSTOM,A,V1C,17,1,2.623741,4.335140,0.280033,0.639258,1,V,C,V
2,CUSTOM,A,V1D,17,2,2.623741,2.824680,0.212670,-0.272264,1,V,D,V
3,CUSTOM,A,V1E,17,3,2.623741,2.712832,0.236621,0.071764,1,V,E,V
4,CUSTOM,A,V1F,17,4,2.623741,3.207937,0.262368,0.415754,1,V,F,V


In [ ]:
#importing the test data
test_data = pd.read_csv("/content/drive/Shareddrives/2:1 Caitlin & Kimai/Data/Enzyme Stability Prediction/test.csv")
test_data.head()

,seq_id,protein_sequence,pH,data_source
0,31390,VPVNPEPDATSVENVAEKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes
1,31391,VPVNPEPDATSVENVAKKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes
2,31392,VPVNPEPDATSVENVAKTGSGDSQSDPIKADLEVKGQSALPFDVDC...,8,Novozymes
3,31393,VPVNPEPDATSVENVALCTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes
4,31394,VPVNPEPDATSVENVALFTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes


In [ ]:
#amino acid of the sequence
wild_type = "VPVNPEPDATSVENVALKTGSGDSQSDPIKADLEVKGQSALPFDVDCWAILCKGAPNVLQRVNEKTKNSNRDRSGANKGPFKDPQKWGIKALPPKNPSWSAQDFKSPEEYAFASSLQGGTNAILAPVNLASQNSQGGVLNGFYSANKVAQFDPSKPQQTKGTWFQITKFTGAAGPYCKALGSNDKSVCDKNKNIAGDWGFDPAKWAYQYDEKNNKFNYVGK"

In [ ]:
#getting the mutations
import Levenshtein as lv

def get_ddg_values(wild_type, df, pred):
    
    ddg = []
    
    for _, row in df.iterrows():
        protein_sequence = row.protein_sequence
        
        if wild_type == protein_sequence:
            ddg.append(0)
            continue
            
        ops = lv.editops(wild_type, protein_sequence)[0]
        
        if ops[0] == 'replace':
            ddg.append(-pred[(pred.pos - 1 == ops[1]) & (pred.mt_AA == protein_sequence[ops[2]])]['score_ml'].iloc[0])
        else:
            ddg.append(pd.NA)
            
    return ddg

In [ ]:
#applying on the dataset
test_data['tm'] = get_ddg_values(wild_type, test_data, rasp_data)

In [ ]:
test_data.head()

,seq_id,protein_sequence,pH,data_source,tm
0,31390,VPVNPEPDATSVENVAEKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,-0.347394
1,31391,VPVNPEPDATSVENVAKKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,-0.42078
2,31392,VPVNPEPDATSVENVAKTGSGDSQSDPIKADLEVKGQSALPFDVDC...,8,Novozymes,<NA>
3,31393,VPVNPEPDATSVENVALCTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,-0.901684
4,31394,VPVNPEPDATSVENVALFTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,-0.349533


In [ ]:
#checking null values
test_data['tm'].isnull().sum()

77

In [ ]:
#filling missing values with mean strategy
data1 = test_data.copy()
data1['tm'].fillna(data1['tm'].mean(), inplace = True)

#save the dataframe
data1 = data1[['seq_id','tm']]
data1.to_csv("submission_mean.csv", index = False)

In [ ]:
#filling missing values with median strategy
data2 = test_data.copy()
data2['tm'].fillna(data2['tm'].median(), inplace = True)

#save the dataframe
data2 = data2[['seq_id','tm']]
data2.to_csv("submission_median.csv", index = False)